In [1]:
"""
Load the required modules
"""
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from VisualOdometry import VisualOdometry
from Camera import Camera
import numpy as np
from dataset import Dataset
from itertools import compress
%pylab inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 280
mpl.rcParams['figure.dpi'] = 280

import cv2

ModuleNotFoundError: No module named 'VisualOdometry'

In [37]:
"""
Init object
"""
matcher_parameters = dict(detector='orb',
                          matcher='bf')
path = '/home/cesar/Documents/Projects/Computer_Vision/TFG/Kitti_Dataset/dataset/sequences/00/image_0'
vo = VisualOdometry(matcher_parameters, path)
""" Performs first iteration"""
vo.init_reconstruction(optimize=False)

Shape needed for recoverpose: (732, 2)
Type needed for recoverpose: <class 'numpy.ndarray'>
Type: <class 'numpy.float64'>


# Tracking local map method

In [38]:
vo.kitti.read_image()
previous_image = vo.kitti.image_1.copy()
points = vo.scene.cameras[vo.index].points
print("Number of map points: {}".format(len(vo.scene.structure)))
# Lucas-Kanade optical flow
mask, lk_prev_points, lk_next_points = vo.matcher.lktracker(previous_image, vo.kitti.image_2, points)
print("Tracked points: {}".format(len(lk_next_points)))
# Compute F and E using the tracked points
F = vo.FindFundamentalRansac(lk_next_points, points[mask])
E = vo.E_from_F(F)
# Create a new cam object
pts1 = (np.reshape(points[mask], (len(points[mask]), 2))).T
pts2 = (np.reshape(lk_next_points, (len(lk_next_points), 2))).T
R, t = vo.get_pose(pts1.T, pts2.T,vo.cam.K, E)
cam = Camera()
cam.set_R(R)
cam.set_t(t)
cam.Rt2P(inplace=True)
# Project local map
vo.scene.add_camera(cam)
kp_proj, projected_map = vo.scene.project_local_map(vo.index + 1, as_kp=True)
print(kp_proj[0].pt[1] >1)
kp_proj = [kp for kp in kp_proj if int(kp.pt[0])>0 ]
kp_proj = [kp for kp in kp_proj if int(kp.pt[0]) < 1230 ]
kp_proj = [kp for kp in kp_proj if int(kp.pt[1])>0 ]
kp_proj = [kp for kp in kp_proj if int(kp.pt[1]) < 360 ]
for point in kp_proj:
    print("Point: {}".format(point.pt))
    #print("Point: {}".format())
    

Number of map points: 246
Tracked points: 246
8
[<MapPoint.MapPoint object at 0x7f22123c7470>, <MapPoint.MapPoint object at 0x7f22123c7a90>, <MapPoint.MapPoint object at 0x7f22123c7630>, <MapPoint.MapPoint object at 0x7f22123c7780>, <MapPoint.MapPoint object at 0x7f22123c7320>]
Number of map points: 246
[1]
[ 429.16903998  152.95569926    1.        ]
True
Point: (429.0, 152.0)
Point: (462.0, 166.0)
Point: (445.0, 152.0)
Point: (447.0, 164.0)
Point: (446.0, 142.0)
Point: (536.0, 182.0)
Point: (529.0, 185.0)
Point: (444.0, 181.0)
Point: (421.0, 143.0)
Point: (441.0, 143.0)
Point: (497.0, 180.0)
Point: (489.0, 180.0)
Point: (436.0, 162.0)
Point: (494.0, 190.0)
Point: (496.0, 190.0)
Point: (474.0, 191.0)
Point: (445.0, 164.0)
Point: (437.0, 177.0)
Point: (537.0, 193.0)
Point: (413.0, 157.0)
Point: (394.0, 174.0)
Point: (536.0, 196.0)
Point: (613.0, 173.0)
Point: (428.0, 147.0)
Point: (441.0, 148.0)
Point: (434.0, 148.0)
Point: (423.0, 148.0)
Point: (482.0, 167.0)
Point: (486.0, 137.0)
Poin

In [ ]:
camera1 = vo.scene.cameras[1]

# Crop image

In [ ]:
data = Dataset('/home/cesar/Documents/Projects/Computer_Vision/TFG/Kitti_Dataset/dataset/sequences/00/image_0')
data.read_image()
print("Wide: {}".format(data.image_width))
print("Height: {}".format(data.image_height))

### 1. Find keypoints in the image (in the application this keypoints are the mappoints)

In [ ]:
detector = cv2.ORB_create(2000)
patch_size = 80
def kp_list_to_np( kp_list):
        """ Transforms a list of Keypoints into a numpy ndarray

        :param kp_list: List of KeyPoint_ objects
        :type kp_list: List

        :returns: A Numpy ndarray (nx2) with the KeyPoints coordinates
        :rtype: Numpy ndarray

        """
        temp = [item.pt for item in kp_list]
        return np.asarray(temp)
kp1, desc1 = detector.detectAndCompute(data.image_2, None)
keypoints = kp_list_to_np(kp1)
# Show the ROI where we will extract new keypoints
roi1 = data.crop_image(np.asarray(keypoints[10]), np.array([patch_size, patch_size]), data.image_2, center=True)
plt.scatter(x=[patch_size / 2], y=[patch_size / 2], c='r', s=20)
img = plt.imshow(roi1)

### 2. Load a new image and crop it using as center a keypoint

In [ ]:
data.read_image()
data.read_image()
data.read_image()
roi = data.crop_image(np.asarray(keypoints[10]), np.array([patch_size, patch_size]), data.image_2, center=True)
img = plt.imshow(roi)

plt.show()

## Find new keypoints in this ROI

In [ ]:
kp2, desc2 = detector.detectAndCompute(roi, None)
keypoints2 = kp_list_to_np(kp2)
# Create a vector of descriptors for the keypoint in question using np.tile
desc_kp = np.tile(desc1[0], (len(kp2), 1))
# Match tracked keypoint descriptor with all the new descriptors
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False) 
matches1 = matcher.knnMatch(desc_kp, desc2, 2) 

# Sort them in the order of their distance.
matches = sorted(matches1, key = lambda x:x[0].distance)
# Print best match coordinates
print(keypoints2[matches[0][0].queryIdx])
trackedpoint = keypoints2[matches[0][0].queryIdx]

## Plot in the new roi the tracked keypoint (center of the roi) and the best candidate

Do this steps for all the map points in the local map and discard bad tracked points using Ransac

In [ ]:
img = plt.imshow(roi)
plt.scatter(x=[patch_size / 2, int(trackedpoint[0])], y=[patch_size / 2, int(trackedpoint[1])], c='r', s=20)
plt.show()

In [ ]:
keypoint = cv2.KeyPoint(x=2,y=3, _size=5)